In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from multiprocess import Pool
import numpy as np
import pandas as pd
import os
from os.path import abspath, join
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, auc, plot_roc_curve
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import pickle

there are four training sets
each of which are divided into three feature sets - fs1, fs2, fs3

fs1 - sequence, GC content, kmer weights, with or without fickett score

fs2 - fs1 + pairing probabilities

fs3 - fs2 + conservation scores

the pipeline is divided into three main parts:
1) dataset curation with the basic features
2) calculation of kmer weights and recording them in tuples (key, weight, frequency)
3) training the random forest classifier

In [2]:
abspath("training_1")
#the balanced dataset is loaded from each feature set-fickett score combination
flanks_down = pd.read_csv("fs1/w_fickett/down_fickett")
#w_fickett -> with fickett
#wo_fickett -> without fickett


FileNotFoundError: [Errno 2] File fs1/w_fickett/down_fickett does not exist: 'fs1/w_fickett/down_fickett'

In [ ]:
flanks_down.info()

kmer weights are calculated and saved in tuples
for every kmer-sequence combination a tuple is created where
key: if the kmer is present in the sequence or not, [1,0]
weight: the normalized kmer weight
frequency: times the kmer occurs

In [ ]:
#a multilabel binarizer is employed to create a feature from each of the tuple elements
cmlb=pd.read_hdf("fs1/cmlb5", key="mlb")

In [ ]:
#the final training set is created combining all the features
features_combined = pd.concat([flanks_down, cmlb], axis=1, copy=False)

In [ ]:
#we split the training set 80-20 for the classifier to train on

features_combined.drop(['gene_id'], 1, inplace=True)

#for phatscons/fs3
#val = {'cons_1':0, 'cons_2':0, 'cons_3':0, 'cons_4':0, 'cons_5':0, 'cons_6':0, 
#'cons_7':0, 'cons_8':0, 'cons_9':0, 'cons_10':0, 'cons_11':0, 'cons_12':0, 'cons_13':0, 
#'cons_14':0, 'cons_15':0, 'cons_16':0, 'cons_17':0, 'cons_18':0, 'cons_19':0, 'cons_20':0}
#features_combined_3.fillna(value=val, inplace=True)


ncrna_train, ncrna_test = train_test_split(features_combined_3, test_size=0.2, random_state=42)

ncrna_test_target=ncrna_test[["target"]]
ncrna_test=ncrna_test.drop(["target"],1)
print (ncrna_test_target.shape)
print (ncrna_test.shape)

ncrna_train_target=ncrna_train[["target"]]
ncrna_train=ncrna_train.drop(["target"],1)
print (ncrna_train_target.shape)
print (ncrna_train.shape)

#separating numerical features and categorical features
num_feat=[]
for x in ncrna_train.columns[1:]:
    num_feat.append(x)

cat_feat = ["seq"]

#encoding the sequence feature
#scaling the rest of the features
#and creating a ColumnTransformer object
cct = ColumnTransformer([
    ('oe', OrdinalEncoder(), cat_feat),
    ('num', StandardScaler(), num_feat)
    ], remainder='passthrough')

#fitting the split training set and test set
X_train = cct.fit_transform(ncrna_train)
X_test = cct.fit_transform(ncrna_test)

#encoding the target labels
ordinal = OrdinalEncoder()
y_train = ordinal.fit_transform(ncrna_train_target)
y_test = ordinal.fit_transform(ncrna_test_target)

y_train = np.ravel(y_train)

print (y_train.shape)
print (y_test.shape)

print (X_train.shape)
print (X_test.shape)

In [ ]:
#training the random forest classifier with default params
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
print (f'accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
#grid search with 10-fold cv
param_grid = {'oob_score': [True, False], 'bootstrap':[True],
              'criterion': ["gini", "entropy"], 'n_estimators':[100,300,500,1000]}
grid_search_def10 = GridSearchCV(RandomForestClassifier(random_state=42),
    param_grid, scoring="accuracy", n_jobs=-1, verbose=1, cv=10)
grid_search_def10.fit(X_train, y_train)

print (f"Grid search cv10 best score: {grid_search_def10.best_score_}")

In [ ]:
#grid search with 5-fold cv starts
param_grid = {'oob_score': [True, False], 'bootstrap': [True],
              'criterion': ["gini", "entropy"], 'n_estimators':[100,300,500,1000]}
grid_search_def5 = GridSearchCV(RandomForestClassifier(random_state=42),
    param_grid, scoring="accuracy", n_jobs=-1, verbose=1, cv=5)
grid_search_def5.fit(X_train, y_train)

print (f"Grid search cv5 best score: {grid_search_def5.best_score_}")

In [ ]:
#model 1 is the best estimator from the grid search with 10-fold cv
model_1 = grid_search_def10.best_estimator_
y_pred = model_1.predict(X_test)
model_1_acc = accuracy_score(y_test, y_pred)
print (f"model 1 accuracy: {model_1_acc}")
y_pred_train = model_1.predict(X_train)
model_1_train_acc = accuracy_score(y_train, y_pred_train)
print (f"model 1 accuracy on training set: {model_1_train_acc}")

In [ ]:
#model 2 is the best estimator from the grid search with 5-fold cv
model_2 = grid_search_def5.best_estimator_
y_pred_5 = model_2.predict(X_test)
model_2_acc = accuracy_score(y_test, y_pred_5)
print (f"model 2 accuracy: {model_2_acc}")
y_pred_train_5 = model_2.predict(X_train)
model_2_train_acc = accuracy_score(y_train, y_pred_train_5)
print (f"model 2 accuracy on training set: {model_2_train_acc}")

In [ ]:
#original model 
y_pred_orig = rnd_clf.predict(X_test)
print (accuracy_score(y_test, y_pred_orig))

In [ ]:
#classification matrix for the original model
print(classification_report(y_test, y_pred))

path = "fs1/w_fickett/results"
if not os.path.isdir(join(path)):
    os.makedirs(join(path))
plt.savefig(join(path,"feat_imp.png"))

disp = plot_confusion_matrix(rnd_clf, X_test, y_test,
                                     display_labels=["lnc","mir","sno"],
                                     cmap=plt.cm.Blues)
disp.ax_.set_title('feature set #1 with fickett score')
print (disp.confusion_matrix)
plt.savefig(join(path,"conf_mat.png"))

In [ ]:
#feature importances
feat_imp = rnd_clf.feature_importances_
importances = pd.Series(feat_imp, index=features_combined.drop(["target"],1).columns)
importances.nlargest(10).plot(kind="barh",figsize=(20,10))

the next section follows the unsupervised learning setup.
k-means clustering of the three RNA classes and PCA with two components.

In [ ]:
#k-means clustering of the three classes
km = KMeans (n_clusters=3, init="k-means++", random_state=42).fit(X_train)

y = km.predict(X_test)

print (f'number of iterations : {km.n_iter_}\n')

#labels : {0: lnc, 1: mir, 2:sno}
#training set
print(f'predicted classes on the training set: {Counter(km.labels_)}\n')
print(f'actual classes on the trainig set: {Counter(y_train)}\n')
print(f'accuracy training set: {accuracy_score(y_train, km.labels_)}\n')
print(f'number of correctly classified samples: {accuracy_score(y_train, km.labels_, normalize=False)}\n')

#test set
print(f'predicted classes on the test set: {Counter(y)}\n')
print(f'actual classes on the test set: {Counter(y_test)}\n') 
print(f'accuracy test set: {accuracy_score(y_test, y)}')

In [ ]:
#pca
p = PCA(n_components=2, random_state=42)

X= p.fit_transform(X_train)

print(f'explained variance ratio: {p.explained_variance_ratio_}')
print(f'n_compponents: {p.n_components_}')
print(f'features: {p.n_features_}')
print(f'samples: {p.n_samples_}')

fig = plt.figure(figsize=(8,3))
colours = ['navy', 'turquoise', 'darkorange']
targets = ['lnc', 'mir', 'sno']
lw=2
for colour, i, target_name in zip(colours, [0, 1, 2], targets):
    plt.scatter(X[y_train == i, 0], X[y_train == i, 1], color=colour, alpha=.8, #lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)